In [1]:
%autosave 0

Autosave disabled


In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

  Using cached grpcio-1.42.0-cp39-cp39-win_amd64.whl (3.3 MB)
  Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl (37 kB)
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.50.0
    Uninstalling grpcio-1.50.0:
      Successfully uninstalled grpcio-1.50.0


In [2]:
!pip install keras-image-helper

  Using cached keras_image_helper-0.0.1-py3-none-any.whl (4.6 kB)
     ---------------------------------------- 2.5/2.5 MB 3.7 MB/s eta 0:00:00


In [3]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [17]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [18]:
from keras_image_helper import create_preprocessor

In [19]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [20]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [21]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [22]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [23]:
pb_response = stub.Predict(pb_request, timeout=50.0)

In [24]:
preds = pb_response.outputs['dense_7'].float_val

In [25]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [26]:
dict(zip(classes, preds))

{'dress': -1.87986421585083,
 'hat': -4.756310939788818,
 'longsleeve': -2.359532356262207,
 'outwear': -1.08926522731781,
 'pants': 9.903783798217773,
 'shirt': -2.826178789138794,
 'shoes': -3.648310899734497,
 'shorts': 3.241154670715332,
 'skirt': -2.6120948791503906,
 't-shirt': -4.852035045623779}